# 🚀 Risk Model Pipeline - Master End-to-End Notebook

Bu notebook baştan sona tüm pipeline'ı çalıştırır:
- Kurulum (pip install)
- Data hazırlama
- Veri sözlüğü oluşturma
- Model eğitimi
- Kalibrasyon
- Skorlama
- Raporlama

**NOT:** Tüm hücreler sırayla çalıştırılmalıdır.

## 📦 BÖLÜM 1: KURULUM VE IMPORT

In [ ]:
# 1.1 - Paket Kurulumu (ilk kurulum için)
# NOT: Eğer zaten kuruluysa bu hücreyi atlayabilirsiniz

# GitHub'dan kurulum
!pip install git+https://github.com/selimoksuz/risk-model-pipeline.git

# Veya lokal kurulum (eğer clone yaptıysanız)
# !pip install -e ../  # notebook klasöründen bir üst dizin

print("✅ Paket kurulumu tamamlandı!")

In [ ]:
# 1.2 - Gerekli kütüphaneleri import et
import pandas as pd
import numpy as np
import warnings
import os
import sys
import json
import joblib
from datetime import datetime, timedelta
from pathlib import Path

warnings.filterwarnings('ignore')

# Pipeline imports
try:
    from risk_pipeline.pipeline16 import RiskModelPipeline, Config
    from risk_pipeline.utils.pipeline_runner import run_pipeline_from_dataframe
    from risk_pipeline.utils.scoring import score_data, load_model_artifacts
    print("✅ Pipeline modülleri başarıyla import edildi (pip install)")
except ImportError:
    # Lokal import (eğer pip install yapmadıysanız)
    sys.path.append('..')
    from src.risk_pipeline.pipeline16 import RiskModelPipeline, Config
    from src.risk_pipeline.utils.pipeline_runner import run_pipeline_from_dataframe
    from src.risk_pipeline.utils.scoring import score_data, load_model_artifacts
    print("✅ Pipeline modülleri başarıyla import edildi (lokal)")

print(f"Python version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

## 📊 BÖLÜM 2: VERİ HAZIRLAMA

In [ ]:
# 2.1 - Ana veri setini oluştur
np.random.seed(42)  # Tekrarlanabilirlik için

# Parametreler
n_train = 10000  # Eğitim verisi boyutu
n_calibration = 2000  # Kalibrasyon verisi boyutu
n_scoring = 3000  # Skorlama verisi boyutu

print("📊 Veri setleri oluşturuluyor...")

# Yardımcı fonksiyon: Veri seti oluştur
def create_dataset(n_samples, start_date, id_prefix, target_rate=0.2, add_missing=True):
    """Risk modeli için sentetik veri oluştur"""
    
    df = pd.DataFrame({
        # ID ve tarih
        'app_id': [f'{id_prefix}_{i:06d}' for i in range(n_samples)],
        'app_dt': pd.date_range(start_date, periods=n_samples, freq='D'),
        
        # Target
        'target': np.random.binomial(1, target_rate, n_samples),
        
        # Numerik özellikler
        'yas': np.random.randint(18, 70, n_samples),
        'gelir': np.random.lognormal(10, 0.5, n_samples),
        'kredi_skoru': np.random.normal(650, 100, n_samples).clip(300, 850),
        'borc_tutari': np.random.exponential(30000, n_samples),
        'kredi_tutari': np.random.exponential(50000, n_samples),
        'calisma_suresi': np.random.exponential(5, n_samples).clip(0, 40),
        'bagimlı_sayisi': np.random.poisson(1.5, n_samples).clip(0, 6),
        'ev_deger': np.random.lognormal(12, 0.8, n_samples),
        
        # Kategorik özellikler
        'egitim': np.random.choice(['Ilkokul', 'Lise', 'Lisans', 'Y.Lisans', 'Doktora'], 
                                   n_samples, p=[0.1, 0.3, 0.35, 0.2, 0.05]),
        'istihdam': np.random.choice(['Maasli', 'Serbest', 'Emekli', 'Ogrenci', 'Issiz'], 
                                     n_samples, p=[0.5, 0.25, 0.1, 0.05, 0.1]),
        'medeni_hal': np.random.choice(['Bekar', 'Evli', 'Bosanmis', 'Dul'], 
                                       n_samples, p=[0.3, 0.5, 0.15, 0.05]),
        'konut_durumu': np.random.choice(['Kendi', 'Kira', 'Aile', 'Lojman'], 
                                         n_samples, p=[0.4, 0.35, 0.2, 0.05]),
        'sehir_tipi': np.random.choice(['Buyuksehir', 'Sehir', 'Ilce', 'Koy'], 
                                       n_samples, p=[0.4, 0.3, 0.2, 0.1]),
        'bolge': np.random.choice(['Marmara', 'Ege', 'Akdeniz', 'IC_Anadolu', 'Karadeniz', 'Dogu', 'GDogu'], 
                                 n_samples, p=[0.3, 0.15, 0.15, 0.15, 0.1, 0.1, 0.05]),
        'sektor': np.random.choice(['Kamu', 'Ozel', 'Serbest'], n_samples, p=[0.3, 0.5, 0.2]),
        'cinsiyet': np.random.choice(['E', 'K'], n_samples, p=[0.52, 0.48])
    })
    
    # Eksik değerler ekle (gerçekçilik için)
    if add_missing:
        missing_cols = ['calisma_suresi', 'bagimlı_sayisi', 'medeni_hal', 'sektor']
        for col in missing_cols:
            missing_idx = np.random.choice(df.index, size=int(0.05 * len(df)), replace=False)
            df.loc[missing_idx, col] = np.nan
    
    return df

print("✅ Veri oluşturma fonksiyonu hazır")

In [ ]:
# 2.2 - Veri setlerini oluştur

# Eğitim verisi
train_df = create_dataset(
    n_samples=n_train,
    start_date='2022-01-01',
    id_prefix='TRAIN',
    target_rate=0.15  # %15 default rate
)

# Kalibrasyon verisi (biraz farklı dağılım)
calibration_df = create_dataset(
    n_samples=n_calibration,
    start_date='2023-07-01',
    id_prefix='CAL',
    target_rate=0.18  # Biraz daha yüksek default rate
)

# Skorlama verisi (bazıları target'sız)
scoring_df = create_dataset(
    n_samples=n_scoring,
    start_date='2023-10-01',
    id_prefix='SCORE',
    target_rate=0.20
)

# Skorlama verisinin bir kısmını target'sız yap
no_target_idx = np.random.choice(scoring_df.index, size=int(0.6 * len(scoring_df)), replace=False)
scoring_df.loc[no_target_idx, 'target'] = np.nan

print("📊 Veri Setleri Özeti:")
print("="*60)
print(f"1. Eğitim Verisi:")
print(f"   - Shape: {train_df.shape}")
print(f"   - Target rate: {train_df['target'].mean():.2%}")
print(f"   - Tarih aralığı: {train_df['app_dt'].min()} - {train_df['app_dt'].max()}")

print(f"\n2. Kalibrasyon Verisi:")
print(f"   - Shape: {calibration_df.shape}")
print(f"   - Target rate: {calibration_df['target'].mean():.2%}")
print(f"   - Tarih aralığı: {calibration_df['app_dt'].min()} - {calibration_df['app_dt'].max()}")

print(f"\n3. Skorlama Verisi:")
print(f"   - Shape: {scoring_df.shape}")
print(f"   - Target olan: {(~scoring_df['target'].isna()).sum()} kayıt")
print(f"   - Target olmayan: {scoring_df['target'].isna().sum()} kayıt")
print(f"   - Tarih aralığı: {scoring_df['app_dt'].min()} - {scoring_df['app_dt'].max()}")

print("\n✅ Tüm veri setleri hazır!")

## 📚 BÖLÜM 3: VERİ SÖZLÜĞÜ HAZIRLAMA

In [ ]:
# 3.1 - Veri sözlüğü oluştur

data_dictionary = pd.DataFrame({
    'alan_adi': [
        'yas', 'gelir', 'kredi_skoru', 'borc_tutari', 'kredi_tutari', 
        'calisma_suresi', 'bagimlı_sayisi', 'ev_deger',
        'egitim', 'istihdam', 'medeni_hal', 'konut_durumu', 
        'sehir_tipi', 'bolge', 'sektor', 'cinsiyet'
    ],
    'alan_aciklamasi': [
        'Müşteri yaşı (yıl)',
        'Aylık gelir tutarı (TL)',
        'Kredi risk skoru (300-850)',
        'Mevcut borç tutarı (TL)',
        'Talep edilen kredi tutarı (TL)',
        'Mevcut işyerinde çalışma süresi (yıl)',
        'Bakmakla yükümlü kişi sayısı',
        'Konut değeri (TL)',
        'En yüksek eğitim seviyesi',
        'İstihdam durumu',
        'Medeni durum',
        'Konut sahiplik durumu',
        'Yerleşim yeri tipi',
        'Coğrafi bölge',
        'Çalışılan sektör',
        'Cinsiyet (E/K)'
    ]
})

print("📚 Veri Sözlüğü:")
print("="*80)
print(data_dictionary.to_string(index=False))

# Excel'e de kaydet (opsiyonel)
data_dict_path = 'outputs/data_dictionary.xlsx'
os.makedirs('outputs', exist_ok=True)
data_dictionary.to_excel(data_dict_path, index=False)
print(f"\n✅ Veri sözlüğü kaydedildi: {data_dict_path}")

## ⚙️ BÖLÜM 4: PIPELINE KONFIGÜRASYONU

In [ ]:
# 4.1 - Pipeline konfigürasyonunu oluştur

cfg = Config(
    # Kolon tanımları
    id_col='app_id',
    time_col='app_dt',
    target_col='target',
    
    # Veri bölme ayarları
    use_test_split=True,
    test_size_row_frac=0.2,  # %20 test
    oot_window_months=3,  # 3 aylık OOT penceresi
    
    # Veri sözlüğü ve kalibrasyon
    data_dictionary_df=data_dictionary,  # Veri sözlüğü DataFrame
    calibration_df=calibration_df,  # Kalibrasyon DataFrame
    calibration_method='isotonic',  # Kalibrasyon metodu
    
    # Model ayarları
    cv_folds=5,  # Cross-validation fold sayısı
    random_state=42,
    n_jobs=-1,  # Tüm CPU'ları kullan
    
    # Hyperparameter optimization
    hpo_timeout=120,  # 2 dakika timeout
    hpo_n_trials=20,  # 20 deneme
    
    # Feature engineering
    rare_threshold=0.02,  # %2'den az görülen kategoriler
    psi_threshold=0.20,  # PSI eşiği
    iv_min=0.02,  # Minimum IV değeri
    corr_threshold=0.95,  # Korelasyon eşiği
    
    # Çıktılar
    output_folder='outputs',
    output_excel_path='master_model_report.xlsx',
    log_file='pipeline.log',
    write_artifacts=True,  # Model artifacts kaydet
    write_csv=False,  # CSV yazma (Excel yeterli)
    write_parquet=False  # Parquet yazma
)

print("⚙️  Konfigürasyon Özeti:")
print("="*60)
print(f"Output klasörü: {cfg.output_folder}")
print(f"Excel raporu: {cfg.output_excel_path}")
print(f"Veri sözlüğü: {len(data_dictionary)} değişken tanımı")
print(f"Kalibrasyon: {len(calibration_df)} kayıt")
print(f"CV Folds: {cfg.cv_folds}")
print(f"HPO Trials: {cfg.hpo_n_trials}")
print(f"Random State: {cfg.random_state}")
print("\n✅ Konfigürasyon hazır!")

## 🚀 BÖLÜM 5: PİPELINE ÇALIŞTIRMA

In [ ]:
# 5.1 - Pipeline'ı çalıştır

print("🚀 Pipeline başlatılıyor...")
print("="*80)
start_time = datetime.now()

# Pipeline oluştur ve çalıştır
pipeline = RiskModelPipeline(cfg)
pipeline.run(train_df)

end_time = datetime.now()
duration = (end_time - start_time).total_seconds()

print("="*80)
print(f"✅ Pipeline tamamlandı! (Süre: {duration:.1f} saniye)")
print(f"\nRun ID: {pipeline.cfg.run_id}")
print(f"Best Model: {pipeline.best_model_name_}")
print(f"Final Features: {len(pipeline.final_vars_)} değişken")

## 📈 BÖLÜM 6: MODEL SONUÇLARINI İNCELEME

In [ ]:
# 6.1 - Model performansını incele

print("📈 Model Performans Özeti:")
print("="*80)

if pipeline.models_summary_ is not None:
    # En önemli metrikleri göster
    summary_cols = ['model', 'auc_traincv', 'auc_test', 'auc_oot', 'gini_oot', 'ks_oot']
    available_cols = [col for col in summary_cols if col in pipeline.models_summary_.columns]
    
    print("\nTüm Modellerin Performansı:")
    print(pipeline.models_summary_[available_cols].to_string())
    
    # En iyi model detayı
    best_model_row = pipeline.models_summary_[pipeline.models_summary_['model'] == pipeline.best_model_name_].iloc[0]
    print(f"\n🏆 En İyi Model: {pipeline.best_model_name_}")
    print(f"   - AUC (Train/CV): {best_model_row.get('auc_traincv', 'N/A')}")
    print(f"   - AUC (Test): {best_model_row.get('auc_test', 'N/A')}")
    print(f"   - AUC (OOT): {best_model_row.get('auc_oot', 'N/A')}")
    print(f"   - Gini (OOT): {best_model_row.get('gini_oot', 'N/A')}")
    print(f"   - KS (OOT): {best_model_row.get('ks_oot', 'N/A')}")

In [ ]:
# 6.2 - En iyi model değişkenlerini incele (veri sözlüğü ile)

print("📊 En İyi Model Değişkenleri (İlk 20):")
print("="*80)

if pipeline.best_model_vars_df_ is not None:
    # Gösterilecek kolonlar
    display_cols = ['variable', 'description', 'coef_or_importance', 'variable_group']
    available_cols = [col for col in display_cols if col in pipeline.best_model_vars_df_.columns]
    
    # İlk 20 değişkeni göster
    print(pipeline.best_model_vars_df_[available_cols].head(20).to_string())
    
    # Açıklamaların yüklendiğini kontrol et
    if 'description' in pipeline.best_model_vars_df_.columns:
        has_desc = pipeline.best_model_vars_df_['description'].notna().sum()
        print(f"\n✅ Veri sözlüğünden {has_desc} değişken açıklaması yüklendi")
else:
    print("❌ Model değişkenleri bulunamadı")

In [ ]:
# 6.3 - WOE raporunu incele (monotonic sıralı)

print("📊 WOE Raporu Örneği (İlk 2 Değişken):")
print("="*80)

if pipeline.best_model_woe_df_ is not None:
    # İlk 2 değişken için WOE binlerini göster
    unique_vars = pipeline.best_model_woe_df_['variable'].unique()[:2]
    
    for var in unique_vars:
        var_woe = pipeline.best_model_woe_df_[pipeline.best_model_woe_df_['variable'] == var]
        
        print(f"\n📌 Değişken: {var}")
        
        # Açıklama varsa göster
        if 'variable_description' in var_woe.columns:
            desc = var_woe['variable_description'].iloc[0]
            if desc:
                print(f"   Açıklama: {desc}")
        
        # WOE binleri
        display_cols = ['group', 'bin_from', 'bin_to', 'count', 'event_rate', 'woe']
        available_cols = [col for col in display_cols if col in var_woe.columns]
        
        print("\n   WOE Binleri (event_rate'e göre sıralı):")
        print(var_woe[available_cols].to_string())
        
        # Monotonicity kontrolü
        event_rates = var_woe['event_rate'].values
ame        is_monotonic = all(event_rates[i] <= event_rates[i+1] for i in range(len(event_rates)-1))
        print(f"   Monotonic: {'✅ Evet' if is_monotonic else '❌ Hayır'}")
else:
    print("❌ WOE raporu bulunamadı")

## 🎯 BÖLÜM 7: SKORLAMA

In [ ]:
# 7.1 - Model artifacts'ları yükle

print("📦 Model Artifacts Yükleniyor...")

run_id = pipeline.cfg.run_id
output_folder = pipeline.cfg.output_folder

# Model dosyaları
model_file = f"{output_folder}/best_model_{run_id}.joblib"
features_file = f"{output_folder}/final_vars_{run_id}.json"
woe_file = f"{output_folder}/woe_mapping_{run_id}.json"
calibrator_file = f"{output_folder}/calibrator_{run_id}.pkl"

# Dosyaları kontrol et
print(f"\nDosya Kontrolü:")
print(f"  Model: {'✅' if os.path.exists(model_file) else '❌'}")
print(f"  Features: {'✅' if os.path.exists(features_file) else '❌'}")
print(f"  WOE Mapping: {'✅' if os.path.exists(woe_file) else '❌'}")
print(f"  Calibrator: {'✅' if os.path.exists(calibrator_file) else '❌'}")

# Yükle
if all(os.path.exists(f) for f in [model_file, features_file, woe_file]):
    model = joblib.load(model_file)
    with open(features_file, 'r') as f:
        final_features = json.load(f)
    with open(woe_file, 'r') as f:
        woe_mapping = json.load(f)
    
    # Calibrator (varsa)
    calibrator = None
    if os.path.exists(calibrator_file):
        calibrator = joblib.load(calibrator_file)
    
    print(f"\n✅ Artifacts yüklendi:")
    print(f"  Model tipi: {type(model).__name__}")
    print(f"  Feature sayısı: {len(final_features)}")
    print(f"  WOE değişken sayısı: {len(woe_mapping)}")
    print(f"  Calibrator: {'Var' if calibrator else 'Yok'}")
else:
    print("❌ Bazı dosyalar bulunamadı!")
    model = pipeline.models_[pipeline.best_model_name_]
    final_features = pipeline.final_vars_
    woe_mapping = {}
    for v, vw in pipeline.woe_map.items():
        woe_mapping[v] = vw.__dict__
    calibrator = pipeline.calibrator_ if hasattr(pipeline, 'calibrator_') else None
    print("✅ Pipeline'dan yüklendi")

In [ ]:
# 7.2 - Skorlama yap

print("🎯 Skorlama Başlatılıyor...")
print("="*80)

# Score data fonksiyonunu import et
try:
    from risk_pipeline.utils.scoring import score_data
except:
    from src.risk_pipeline.utils.scoring import score_data

# Skorlama yap
scoring_results = score_data(
    scoring_df=scoring_df,
    model=model,
    final_features=final_features,
    woe_mapping=woe_mapping,
    calibrator=calibrator,
    training_scores=None,  # PSI hesaplama için (şimdilik None)
    feature_mapping=None
)

print("\n📊 Skorlama Sonuçları:")
print(f"  Toplam skorlanan: {scoring_results['n_total']:,}")
print(f"  Target'lı kayıt: {scoring_results['n_with_target']:,}")
print(f"  Target'sız kayıt: {scoring_results['n_without_target']:,}")

# Target'lı kayıtlar için performans
if 'with_target' in scoring_results and scoring_results['with_target']:
    print(f"\n📈 Performans (Target'lı Kayıtlar):")
    print(f"  AUC: {scoring_results['with_target']['auc']:.4f}")
    print(f"  Gini: {scoring_results['with_target']['gini']:.4f}")
    print(f"  KS: {scoring_results['with_target']['ks']:.4f}")

# Skor dağılımı
if 'scores' in scoring_results:
    scores = np.array(scoring_results['scores'])
    print(f"\n📊 Skor Dağılımı:")
    print(f"  Min: {scores.min():.4f}")
    print(f"  Q1: {np.percentile(scores, 25):.4f}")
    print(f"  Median: {np.median(scores):.4f}")
    print(f"  Q3: {np.percentile(scores, 75):.4f}")
    print(f"  Max: {scores.max():.4f}")
    print(f"  Mean: {scores.mean():.4f}")
    print(f"  Std: {scores.std():.4f}")

print("\n✅ Skorlama tamamlandı!")

## 📊 BÖLÜM 8: EXCEL RAPORU İNCELEME

In [ ]:
# 8.1 - Excel raporunu kontrol et

excel_path = os.path.join(cfg.output_folder, cfg.output_excel_path)

print("📊 Excel Raporu Kontrolü:")
print("="*80)

if os.path.exists(excel_path):
    print(f"✅ Excel dosyası bulundu: {excel_path}")
    print(f"   Boyut: {os.path.getsize(excel_path) / 1024:.1f} KB")
    
    # Excel'deki sheet'leri listele
    excel_file = pd.ExcelFile(excel_path)
    sheets = excel_file.sheet_names
    
    print(f"\n📋 Toplam {len(sheets)} sheet bulundu:")
    print("-"*60)
    
    # Sheet'leri kategorize et
    main_sheets = ['final_vars', 'best_name', 'models_summary', 'best_model_vars_df', 'best_model_woe_df']
    performance_sheets = ['ks_info_traincv', 'ks_info_test', 'ks_info_oot', 'gini_summary']
    feature_sheets = ['top20_iv_df', 'top50_univariate', 'psi_summary', 'psi_dropped_features']
    correlation_sheets = ['correlation_matrix', 'correlation_clusters', 'corr_dropped']
    
    print("\n🔹 Ana Raporlar:")
    for sheet in main_sheets:
        if sheet in sheets:
            print(f"   ✅ {sheet}")
    
    print("\n🔹 Performans Raporları:")
    for sheet in performance_sheets:
        if sheet in sheets:
            print(f"   ✅ {sheet}")
    
    print("\n🔹 Feature Raporları:")
    for sheet in feature_sheets:
        if sheet in sheets:
            print(f"   ✅ {sheet}")
    
    print("\n🔹 Korelasyon Raporları:")
    for sheet in correlation_sheets:
        if sheet in sheets:
            print(f"   ✅ {sheet}")
    
    # Diğer sheet'ler
    other_sheets = [s for s in sheets if s not in main_sheets + performance_sheets + feature_sheets + correlation_sheets]
    if other_sheets:
        print("\n🔹 Diğer Raporlar:")
        for sheet in other_sheets:
            print(f"   ✅ {sheet}")
    
    # Veri sözlüğü kontrolü
    print("\n📚 Veri Sözlüğü Entegrasyonu:")
    if 'best_model_vars_df' in sheets:
        best_vars = pd.read_excel(excel_path, sheet_name='best_model_vars_df')
        if 'description' in best_vars.columns:
            has_desc = best_vars['description'].notna().sum()
            print(f"   ✅ {has_desc}/{len(best_vars)} değişken açıklaması mevcut")
        else:
            print("   ❌ Açıklama kolonu bulunamadı")
    
    # WOE monotonic kontrolü
    print("\n📈 WOE Monotonic Sıralama:")
    if 'best_model_woe_df' in sheets:
        woe_df = pd.read_excel(excel_path, sheet_name='best_model_woe_df')
        if 'bin_from' in woe_df.columns and 'bin_to' in woe_df.columns:
            print("   ✅ bin_from/bin_to kolonları mevcut")
        if 'variable_description' in woe_df.columns:
            print("   ✅ Değişken açıklamaları mevcut")
        
        # Monotonic kontrol
        first_var = woe_df['variable'].iloc[0] if len(woe_df) > 0 else None
        if first_var:
            var_woe = woe_df[woe_df['variable'] == first_var]
            event_rates = var_woe['event_rate'].values
            is_mono = all(event_rates[i] <= event_rates[i+1] for i in range(len(event_rates)-1))
            print(f"   {'✅' if is_mono else '❌'} İlk değişken monotonic sıralı")
    
else:
    print(f"❌ Excel dosyası bulunamadı: {excel_path}")

print("\n" + "="*80)
print("✅ Excel raporu kontrolü tamamlandı!")

## 🎯 BÖLÜM 9: ÖZET VE SONUÇLAR

In [ ]:
# 9.1 - Genel özet

print("🎯 PIPELINE ÇALIŞTIRMA ÖZETİ")
print("="*80)

print("\n📊 Veri:")
print(f"  • Eğitim: {train_df.shape[0]:,} kayıt")
print(f"  • Kalibrasyon: {calibration_df.shape[0]:,} kayıt")
print(f"  • Skorlama: {scoring_df.shape[0]:,} kayıt")
print(f"  • Toplam değişken: {len(train_df.columns) - 3} (ID, tarih, target hariç)")

print("\n🏆 Model:")
print(f"  • En iyi model: {pipeline.best_model_name_}")
print(f"  • Final değişken sayısı: {len(pipeline.final_vars_)}")
print(f"  • Kalibrasyon: {'Uygulandı ✅' if pipeline.calibrator_ is not None else 'Uygulanmadı ❌'}")

if pipeline.models_summary_ is not None and pipeline.best_model_name_:
    best_row = pipeline.models_summary_[pipeline.models_summary_['model'] == pipeline.best_model_name_].iloc[0]
    print(f"\n📈 Performans:")
    print(f"  • AUC (OOT): {best_row.get('auc_oot', 'N/A')}")
    print(f"  • Gini (OOT): {best_row.get('gini_oot', 'N/A')}")
    print(f"  • KS (OOT): {best_row.get('ks_oot', 'N/A')}")

print(f"\n📁 Çıktılar:")
print(f"  • Excel raporu: {cfg.output_folder}/{cfg.output_excel_path}")
print(f"  • Log dosyası: {cfg.output_folder}/{cfg.log_file}")
print(f"  • Model artifacts: {cfg.output_folder}/best_model_{pipeline.cfg.run_id}.joblib")
print(f"  • Run ID: {pipeline.cfg.run_id}")

print("\n✨ Özellikler:")
print(f"  • Veri sözlüğü entegrasyonu ✅")
print(f"  • WOE monotonic sıralama ✅")
print(f"  • DataFrame kalibrasyon desteği ✅")
print(f"  • Gelişmiş Excel raporlama ✅")
print(f"  • Otomatik feature engineering ✅")

print("\n" + "="*80)
print("🎉 TÜM İŞLEMLER BAŞARIYLA TAMAMLANDI!")
print("="*80)

## 💾 BÖLÜM 10: SONUÇLARI KAYDETME

In [ ]:
# 10.1 - Önemli sonuçları kaydet

# Sonuçları bir dictionary'de topla
results_summary = {
    'run_id': pipeline.cfg.run_id,
    'run_date': datetime.now().isoformat(),
    'best_model': pipeline.best_model_name_,
    'n_final_features': len(pipeline.final_vars_),
    'final_features': pipeline.final_vars_,
    'n_train': train_df.shape[0],
    'n_calibration': calibration_df.shape[0],
    'n_scoring': scoring_df.shape[0],
    'calibration_applied': pipeline.calibrator_ is not None,
    'output_folder': cfg.output_folder,
    'excel_report': cfg.output_excel_path
}

# Performans metrikleri ekle
if pipeline.models_summary_ is not None and pipeline.best_model_name_:
    best_row = pipeline.models_summary_[pipeline.models_summary_['model'] == pipeline.best_model_name_].iloc[0]
    results_summary['performance'] = {
        'auc_traincv': float(best_row.get('auc_traincv', 0)),
        'auc_test': float(best_row.get('auc_test', 0)),
        'auc_oot': float(best_row.get('auc_oot', 0)),
        'gini_oot': float(best_row.get('gini_oot', 0)),
        'ks_oot': float(best_row.get('ks_oot', 0))
    }

# JSON olarak kaydet
results_file = f"{cfg.output_folder}/run_summary_{pipeline.cfg.run_id}.json"
with open(results_file, 'w') as f:
    json.dump(results_summary, f, indent=2)

print(f"💾 Sonuç özeti kaydedildi: {results_file}")

# Skorlama sonuçlarını da kaydet
if 'scoring_results' in locals():
    scoring_file = f"{cfg.output_folder}/scoring_results_{pipeline.cfg.run_id}.json"
    
    # Numpy array'leri liste'ye çevir
    scoring_save = {
        'n_total': scoring_results['n_total'],
        'n_with_target': scoring_results['n_with_target'],
        'n_without_target': scoring_results['n_without_target']
    }
    
    if 'with_target' in scoring_results:
        scoring_save['with_target_metrics'] = scoring_results['with_target']
    
    with open(scoring_file, 'w') as f:
        json.dump(scoring_save, f, indent=2)
    
    print(f"💾 Skorlama sonuçları kaydedildi: {scoring_file}")

print("\n✅ Tüm sonuçlar başarıyla kaydedildi!")

---

## 🎉 TEBRİKLER!

End-to-end pipeline başarıyla tamamlandı. Oluşturulan dosyalar:

1. **Excel Raporu**: `outputs/master_model_report.xlsx` - Tüm raporlar tek dosyada
2. **Model**: `outputs/best_model_[run_id].joblib`
3. **WOE Mapping**: `outputs/woe_mapping_[run_id].json`
4. **Final Features**: `outputs/final_vars_[run_id].json`
5. **Calibrator**: `outputs/calibrator_[run_id].pkl`
6. **Log Dosyası**: `outputs/pipeline.log`
7. **Sonuç Özeti**: `outputs/run_summary_[run_id].json`

### Sonraki Adımlar:
- Diğer simulation notebook'larını deneyin
- Excel raporunu detaylı inceleyin
- Model'i production'a deploy edin